In [18]:
import pandas as pd
import os

In [19]:
import pickle

In [20]:
topics_df = pd.read_csv('../joint_clean_data/topics.csv', index_col=0)

In [21]:
topics_df.head()

,goal_id,goal_domain_original,goal_name_type_original,goal_domain_LoW,goal_name_type_LoW,goal_domain_no_noise,goal_name_type_no_noise,goal_domain_mean_pos,goal_name_type_mean_pos,remove,...,label_attractor_hard_skill,label_attractor_soft_skill,label_attractor_tool,label_attractor_community,label_attractor_subjectivity,label_attractor_habits,label_attractor_career,label_attractor_fixing,label_attractor_art,label_attractor_health
0,19,творчество и создание нового изобразительное и...,не отстать от поезда современности получить зн...,"['творчество', 'и', 'создание', 'нового', 'изо...","['не', 'отстать', 'от', 'поезда', 'современнос...",творчество создание новое изобразительный иску...,отстать поезд современность получить знание но...,творчество создание новое изобразительный иску...,отстать поезд современность получить знание но...,0,...,1,1,0,1,0,1,1,1,0,0.0
1,20,универсальные компетенции гибкие навыки soft ...,самореализация получить знания в новой области...,"['универсальные', 'компетенции', '', 'гибкие',...","['самореализация', 'получить', 'знания', 'в', ...",универсальный компетенция гибкий навык soft s...,самореализация получить знание новый область ...,универсальный компетенция гибкий навык навык о...,самореализация получить знание новый область п...,0,...,1,0,1,0,1,0,1,0,1,0.0
2,21,иностранные языки гуманитарные знания и навык...,улучшенный образ жизни включиться в новый прое...,"['иностранные', 'языки', '', 'гуманитарные', '...","['улучшенный', 'образ', 'жизни', 'включиться',...",иностранный язык гуманитарный знание навык ин...,улучшить образ жизнь включиться новый проект д...,иностранный язык гуманитарный знание навык ино...,образ жизнь включиться новый проект деятельность,0,...,1,1,0,0,0,0,0,1,0,0.0
3,22,прикладные знания и навыки ручной труд информ...,обновление устаревшей информации развить имеющ...,"['прикладные', 'знания', 'и', 'навыки', '', 'р...","['обновление', 'устаревшей', 'информации', 'ра...",прикладной знание навык ручной труд информаци...,обновление устаревший информация развить иметь...,прикладной знание навык ручной труд информация...,обновление устаревший информация развить знани...,0,...,1,0,0,0,1,1,1,1,1,0.0
4,23,универсальные компетенции гибкие навыки soft ...,повышение результативности труда получить знан...,"['универсальные', 'компетенции', '', 'гибкие',...","['повышение', 'результативности', 'труда', 'по...",универсальный компетенция гибкий навык soft s...,повышение результативность труд получить знани...,универсальный компетенция гибкий навык планиро...,повышение результативность труд получить знани...,0,...,1,0,0,0,0,0,0,0,1,0.0


In [22]:
topics_df.columns

Index(['goal_id', 'goal_domain_original', 'goal_name_type_original',
       'goal_domain_LoW', 'goal_name_type_LoW', 'goal_domain_no_noise',
       'goal_name_type_no_noise', 'goal_domain_mean_pos',
       'goal_name_type_mean_pos', 'remove', 'are_first_steps_known',
       'is_time_certain', 'is_certainly_imagined', 'are_obstackles_expected',
       'topic_words', 'goal_words', 'topic_letters', 'goal_letters',
       'topic_aver_word_len', 'goal_aver_word_len', 'goal_verbs_counter',
       'goal_nouns_counter', 'goal_numr_counter', 'goal_adj_counter',
       'goal_digit_counter', 'goal_name', 'goal_name_LoW',
       'goal_name_clean_LoW', 'goal_name_clean_NV_LoW',
       'label_attractor_knowledge', 'label_attractor_hard_skill',
       'label_attractor_soft_skill', 'label_attractor_tool',
       'label_attractor_community', 'label_attractor_subjectivity',
       'label_attractor_habits', 'label_attractor_career',
       'label_attractor_fixing', 'label_attractor_art',
       'label_at

In [23]:
topics = topics_df.copy()

In [24]:
topics = topics[topics['remove'] == 0]

In [25]:
topics.drop(columns=['remove'], inplace=True)

In [26]:
topic_cols = ['label_attractor_knowledge',
       'label_attractor_hard_skill', 'label_attractor_soft_skill',
       'label_attractor_tool', 'label_attractor_community',
       'label_attractor_subjectivity', 'label_attractor_habits',
       'label_attractor_career', 'label_attractor_fixing',
       'label_attractor_art', 'label_attractor_health']

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

In [28]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
for topic in topic_cols:
    topics_text_vectors = topics[['goal_name_clean_NV_LoW', topic]]
    topics_text_vectors.dropna(inplace=True)
    text = topics_text_vectors['goal_name_clean_NV_LoW']
    y = topics_text_vectors[topic]
    tfidf = TfidfVectorizer()
    X = tfidf.fit_transform(text)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
    xgb = XGBClassifier()
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)
    name = topic.split('_')[-1]
    with open(f'../models/topics_goal_name/goal_{name}_vect_xgb.pkl', 'wb') as f:
        pickle.dump(xgb, f)

In [30]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

In [34]:
def create_model():
    vec_size = 100

    model = Sequential()
    model.add(Embedding(vocab_size, vec_size, input_length=max_len))

    model.add(Conv1D(32, 2, activation='relu'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(0.5))

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(16, activation='relu'))

    model.add(GlobalMaxPooling1D())

    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [35]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

In [36]:
for topic in topic_cols:
    topics_text_vectors = topics[['goal_name_clean_NV_LoW', topic]]
    topics_text_vectors.dropna(inplace=True)
    text = topics_text_vectors['goal_name_clean_NV_LoW']
    y = topics_text_vectors[topic]
    token = Tokenizer()
    token.fit_on_texts(text)
    vocab_size = len(token.word_index) + 1
    encoded_text = token.texts_to_sequences(text)
    max_len = int()
    for i in encoded_text:
        len_ = len(i)
        if len_ > max_len:
            max_len = len_
    X = pad_sequences(encoded_text, maxlen=max_len, padding='post')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
    model = create_model()
    X_train = np.asarray(X_train)
    y_train = np.asarray(y_train)
    X_test= np.asarray(X_test)
    y_test = np.asarray(y_test)
    model.fit(X_train, y_train, epochs=600, validation_data=(X_test, y_test), callbacks=[early_stop])
    name = topic.split('_')[-1]
    model.save(f"../models/topics_goal_name/goal_{name}_vect_nn.h5")

Train on 14265 samples, validate on 3567 samples
Epoch 1/600
14265/14265 [==============================] - 7s 458us/sample - loss: 0.3612 - accuracy: 0.8860 - val_loss: 0.4519 - val_accuracy: 0.8946
Epoch 2/600
14265/14265 [==============================] - 6s 408us/sample - loss: 0.3409 - accuracy: 0.8945 - val_loss: 0.4486 - val_accuracy: 0.8946
Epoch 3/600
14265/14265 [==============================] - 6s 423us/sample - loss: 0.3398 - accuracy: 0.8945 - val_loss: 0.4189 - val_accuracy: 0.8946
Epoch 4/600
14265/14265 [==============================] - 6s 436us/sample - loss: 0.3388 - accuracy: 0.8945 - val_loss: 0.4283 - val_accuracy: 0.8946
Epoch 5/600
14265/14265 [==============================] - 6s 417us/sample - loss: 0.3382 - accuracy: 0.8945 - val_loss: 0.4440 - val_accuracy: 0.8946
Epoch 6/600
14265/14265 [==============================] - 6s 409us/sample - loss: 0.3378 - accuracy: 0.8945 - val_loss: 0.4308 - val_accuracy: 0.8946
Epoch 7/600
14265/14265 [====================

14265/14265 [==============================] - 6s 392us/sample - loss: 0.3438 - accuracy: 0.8569 - val_loss: 0.7238 - val_accuracy: 0.7886
Epoch 20/600
14265/14265 [==============================] - 6s 409us/sample - loss: 0.3329 - accuracy: 0.8590 - val_loss: 0.7969 - val_accuracy: 0.7883
Epoch 21/600
14265/14265 [==============================] - 6s 401us/sample - loss: 0.3306 - accuracy: 0.8618 - val_loss: 0.7712 - val_accuracy: 0.7833
Epoch 22/600
14265/14265 [==============================] - 6s 397us/sample - loss: 0.3231 - accuracy: 0.8627 - val_loss: 0.8179 - val_accuracy: 0.7844
Epoch 23/600
14265/14265 [==============================] - 6s 426us/sample - loss: 0.3211 - accuracy: 0.8640 - val_loss: 0.8902 - val_accuracy: 0.7892
Epoch 24/600
14265/14265 [==============================] - 6s 434us/sample - loss: 0.3159 - accuracy: 0.8663 - val_loss: 0.8399 - val_accuracy: 0.7802
Epoch 25/600
14265/14265 [==============================] - 6s 400us/sample - loss: 0.3166 - accuracy

14265/14265 [==============================] - 6s 402us/sample - loss: 0.2841 - accuracy: 0.9149 - val_loss: 0.3692 - val_accuracy: 0.9148
Epoch 12/600
14265/14265 [==============================] - 6s 405us/sample - loss: 0.2790 - accuracy: 0.9149 - val_loss: 0.3686 - val_accuracy: 0.9148
Epoch 13/600
14265/14265 [==============================] - 6s 397us/sample - loss: 0.2741 - accuracy: 0.9149 - val_loss: 0.3721 - val_accuracy: 0.9148
Epoch 14/600
14265/14265 [==============================] - 6s 403us/sample - loss: 0.2697 - accuracy: 0.9149 - val_loss: 0.3823 - val_accuracy: 0.9148
Epoch 15/600
14265/14265 [==============================] - 5s 382us/sample - loss: 0.2653 - accuracy: 0.9149 - val_loss: 0.3989 - val_accuracy: 0.9148
Epoch 16/600
14265/14265 [==============================] - 6s 391us/sample - loss: 0.2622 - accuracy: 0.9149 - val_loss: 0.3937 - val_accuracy: 0.9148
Epoch 17/600
14265/14265 [==============================] - 6s 393us/sample - loss: 0.2576 - accuracy

14265/14265 [==============================] - 7s 457us/sample - loss: 0.3182 - accuracy: 0.8693 - val_loss: 0.9462 - val_accuracy: 0.7673
Epoch 28/600
14265/14265 [==============================] - 7s 463us/sample - loss: 0.3138 - accuracy: 0.8693 - val_loss: 0.9956 - val_accuracy: 0.7645
Epoch 29/600
14265/14265 [==============================] - 7s 517us/sample - loss: 0.3085 - accuracy: 0.8728 - val_loss: 0.9691 - val_accuracy: 0.7555
Epoch 30/600
14265/14265 [==============================] - 7s 491us/sample - loss: 0.3093 - accuracy: 0.8735 - val_loss: 1.0789 - val_accuracy: 0.7620
Epoch 31/600
14265/14265 [==============================] - 7s 481us/sample - loss: 0.3031 - accuracy: 0.8753 - val_loss: 1.0656 - val_accuracy: 0.7561
Epoch 00031: early stopping
Train on 14265 samples, validate on 3567 samples
Epoch 1/600
14265/14265 [==============================] - 8s 542us/sample - loss: 0.6587 - accuracy: 0.6346 - val_loss: 0.6607 - val_accuracy: 0.6414
Epoch 2/600
14265/14265 [

14265/14265 [==============================] - 6s 450us/sample - loss: 0.2812 - accuracy: 0.9118 - val_loss: 0.4849 - val_accuracy: 0.8570
Epoch 23/600
14265/14265 [==============================] - 7s 462us/sample - loss: 0.2795 - accuracy: 0.9134 - val_loss: 0.4783 - val_accuracy: 0.8595
Epoch 24/600
14265/14265 [==============================] - 7s 475us/sample - loss: 0.2766 - accuracy: 0.9129 - val_loss: 0.4909 - val_accuracy: 0.8559
Epoch 25/600
14265/14265 [==============================] - 6s 451us/sample - loss: 0.2726 - accuracy: 0.9132 - val_loss: 0.4987 - val_accuracy: 0.8584
Epoch 26/600
14265/14265 [==============================] - 6s 454us/sample - loss: 0.2695 - accuracy: 0.9145 - val_loss: 0.4726 - val_accuracy: 0.8632
Epoch 27/600
14265/14265 [==============================] - 7s 458us/sample - loss: 0.2639 - accuracy: 0.9153 - val_loss: 0.5348 - val_accuracy: 0.8537
Epoch 28/600
14265/14265 [==============================] - 7s 504us/sample - loss: 0.2620 - accuracy

14265/14265 [==============================] - 7s 461us/sample - loss: 0.4043 - accuracy: 0.8175 - val_loss: 0.8024 - val_accuracy: 0.7157
Epoch 17/600
14265/14265 [==============================] - 8s 548us/sample - loss: 0.3926 - accuracy: 0.8235 - val_loss: 0.8581 - val_accuracy: 0.7205
Epoch 18/600
14265/14265 [==============================] - 8s 529us/sample - loss: 0.3879 - accuracy: 0.8261 - val_loss: 0.8986 - val_accuracy: 0.7160
Epoch 19/600
14265/14265 [==============================] - 8s 551us/sample - loss: 0.3840 - accuracy: 0.8273 - val_loss: 0.8854 - val_accuracy: 0.7129
Epoch 20/600
14265/14265 [==============================] - 7s 483us/sample - loss: 0.3781 - accuracy: 0.8287 - val_loss: 0.9163 - val_accuracy: 0.7110
Epoch 21/600
14265/14265 [==============================] - 7s 463us/sample - loss: 0.3752 - accuracy: 0.8321 - val_loss: 0.9190 - val_accuracy: 0.7107
Epoch 22/600
14265/14265 [==============================] - 7s 483us/sample - loss: 0.3708 - accuracy

14265/14265 [==============================] - 7s 478us/sample - loss: 0.2198 - accuracy: 0.9488 - val_loss: 0.3303 - val_accuracy: 0.9540
Epoch 2/600
14265/14265 [==============================] - 6s 398us/sample - loss: 0.1895 - accuracy: 0.9541 - val_loss: 0.3557 - val_accuracy: 0.9540
Epoch 3/600
14265/14265 [==============================] - 6s 417us/sample - loss: 0.1883 - accuracy: 0.9541 - val_loss: 0.3438 - val_accuracy: 0.9540
Epoch 4/600
14265/14265 [==============================] - 6s 417us/sample - loss: 0.1884 - accuracy: 0.9541 - val_loss: 0.3506 - val_accuracy: 0.9540
Epoch 5/600
14265/14265 [==============================] - 6s 412us/sample - loss: 0.1887 - accuracy: 0.9541 - val_loss: 0.3408 - val_accuracy: 0.9540
Epoch 6/600
14265/14265 [==============================] - 6s 446us/sample - loss: 0.1876 - accuracy: 0.9541 - val_loss: 0.3141 - val_accuracy: 0.9540
Epoch 7/600
14265/14265 [==============================] - 6s 428us/sample - loss: 0.1891 - accuracy: 0.95